## JSDA scraper using Selenium and Chrome webdriver
### Requirements:
- Chrome browser
- Chrome Webdriver. Download it here:http://chromedriver.chromium.org/
- Python packages:
  - Selenium

### **Note**: 
- You should not use your mouse to click around when the scraper is running!
- And also set the time range of the data before running it.

In [50]:
import datetime as dt

start = dt.datetime.strptime("2018-1-1", "%Y-%m-%d")
end = dt.datetime.strptime("2018-12-31", "%Y-%m-%d")
date_generated = [start + dt.timedelta(days=x) for x in range(0, (end-start).days+1)]

In [49]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

options = Options()
options.binary_location = "yourchromebinarypath"
#options.add_argument('--headless')
options.add_argument("download.default_directory='somepath'")
driver = webdriver.Chrome(options=options, executable_path='yourwebdriverpath')

target_url = 'http://market.jsda.or.jp/html/saiken/kehai/downloadInput.php'
driver.get(target_url)

In [51]:
def AutoClick(datetime):
    try:
        # click radio button '格付けマトリクス'
        driver.find_element_by_css_selector("input[name='dataType'][type='radio'][value='R']").click()

        # select time range
        time = datetime
        driver.find_element_by_xpath(".//select[@name='dlDateYear' and \
                                        @class='inputBold'] \
                                        /option[text()='{}']".format(time[0])).click()
        driver.find_element_by_xpath(".//select[@name='dlDateMon' and \
                                        @class='inputBold'] \
                                        /option[text()='{}']".format(time[1])).click()
        driver.find_element_by_xpath(".//select[@name='dlDateDay' and \
                                        @class='inputBold'] \
                                        /option[text()='{}']".format(time[2])).click()

        # click radio button 'CSV'
        driver.find_element_by_css_selector("input[name='dataForm'][type='radio'][value='csv']").click()

        # wait for 5 secs before downloading
        driver.implicitly_wait(5)

        # Okay, download the file
        driver.find_element_by_css_selector("input[class='inputBold'][type='submit'][name='dl'][value='ダウンロード']").click()
    except NoSuchElementException:
        print (datetime)
    
for d in date_generated:
    dd = d.strftime("%Y-%m-%d").split('-')
    if dd[1][0] == '0':
        dd[1] = dd[1][1]
    if dd[2][0] == '0':
        dd[2] = dd[2][1]
    AutoClick(dd)

In [52]:
import glob, os

names = glob.glob('yourdownloadfolder/R*.csv')
with open ('somepath/2018_jsda.csv', 'w', encoding='shiftjis') as f:
    for idx, n in enumerate(names):
        with open (n, 'r', encoding='shiftjis') as ff:
            cache = ff.read()
        if idx == 0:
            f.write(cache)
        else:
            f.write('\n'+cache)
        os.remove(n)

In [53]:
driver.quit()

In [58]:
with open ('somepath/2018_jsda.csv', 'r', encoding='shiftjis') as f:
    test = f.read()
test = test.split('\n')
dates_t = [x[:x.find(',')] for x in test]